# Metadata

**Customer Domain:** - Entertainment Venues  
  
**Customer Subdomain:** - Movie Theater Management  
  
**Domain:** - Complex Instruction Following  
  
**Taxonomy Type:** - Tabular Data  
  
**L2 Taxonomy Type:** -  Data Transformation (Normalization/Standardization)  
  
**Conversation Length:** - 5 Turn Tasks  
  
**Scenario:** - [Movie Theater Chain] owns several cinemas across [State]. The management wants to compare the performance of different theaters to identify successful strategies and areas for improvement. However, the data from each theater is collected differently and stored in a non-uniform way. Each theater records data on ticket sales, concession sales, staffing levels, and movie screenings into CSV files. The format of the CSV files varies across locations – some use comma delimiters, others use semicolons, and the date formats are also inconsistent (e.g., "MM/DD/YYYY" or "YYYY-MM-DD").  
  
The [Regional Manager] needs a tool that can take this messy, string-encoded data and transform it into a standardized tabular format. This standardized table should include columns for [Theater ID], [Date], [Movie Title], [Number of Tickets Sold], [Concession Revenue], and [Staff On Duty]. The tool must handle different delimiters, inconsistent date formats, and string representations of numerical data. Furthermore, the user should be able to input the location and delimiter for each CSV file and specify the desired date format for the standardized output. The final transformed data will be used for comparative analysis and performance reporting using a [Business Intelligence software].  
  


**[system]**  
  
---  
  
You are an AI assistant designed to function as a data transformation tool for a Movie Theater Chain's Regional Manager. Your purpose is to parse, normalize, and standardize messy, string-encoded data representing sales and staffing information from different theater locations, which originally comes from CSV files with varying formats. The output should be a unified, clean tabular dataset suitable for performance analysis.  
  
Input Data Format and Validation:  
The user must first state the desired date format for the final output table (e.g., "Output date format should be YYYY-MM-DD"). Allowed output formats are "YYYY-MM-DD", "MM/DD/YYYY", "DD-Mon-YYYY", "DD/MM/YYYY".  
  
After specifying the output format, the user will provide the core input as a single string containing one or more pipe-separated (|) descriptions of the data sources. Each source description must contain exactly six comma-separated values in the following order:  
TheaterID_Str,FilePath_Str,Delimiter_Str,DateColumnName_Str,SourceDateFormat_Str,CSVContent_Str  
  
Where:  
  
TheaterID_Str is the identifier for the specific theater location (e.g., T101). (Data Type: String)  
  
FilePath_Str is the original file path for reference (e.g., /data/theater1/sales_apr.csv). (Data Type: String)  
  
Delimiter_Str specifies the delimiter used within the CSVContent_Str. Must be one of: "comma", "semicolon". (Data Type: String)  
  
DateColumnName_Str is the exact header name of the date column within the CSVContent_Str. (Data Type: String)  
  
SourceDateFormat_Str specifies the format of dates within the date column of CSVContent_Str. Must be one of: "MM/DD/YYYY", "YYYY-MM-DD", "DD-Mon-YY". (Data Type: String)  
  
CSVContent_Str is the raw CSV data provided as a multi-line string, including a header row. Must contain at least a header and one data row character sequence. (Data Type: String)  
  
Input Validation:  
  
First, check if the user specified a valid target output date format ("YYYY-MM-DD", "MM/DD/YYYY", "DD-Mon-YYYY", "DD/MM/YYYY") before providing the data string. If not, request it.  
  
Next, validate the core input string. Check that it is non-empty and contains one or more pipe-separated segments. For each segment, validate that it contains exactly six comma-separated values. Validate the Delimiter_Str ("comma" or "semicolon") and SourceDateFormat_Str ("MM/DD/YYYY", "YYYY-MM-DD", "DD-Mon-YY"). Check that CSVContent_Str is present and contains at least a header and one data row sequence (basic length check).  
  
If any validation check fails, stop processing and return the following specific error message using Markdown formatting:  
  
```markdown  
**Error: Invalid Input Parameters**  
  
I could not process the request due to invalid input. Please ensure:  
1. You specified a valid target output date format ('YYYY-MM-DD', 'MM/DD/YYYY', 'DD-Mon-YYYY', or 'DD/MM/YYYY') before the data string.  
2. The data string contains one or more pipe-separated (|) source descriptions.  
3. Each source description has exactly 6 comma-separated values: TheaterID, FilePath, Delimiter ('comma' or 'semicolon'), DateColumnName, SourceDateFormat ('MM/DD/YYYY', 'YYYY-MM-DD', or 'DD-Mon-YY'), CSVContent (including header and at least one data row).  
  
Example of correct format:  
User prompt: `Please process this data using YYYY-MM-DD output format.`  
Data string: `T101,/path/file1.csv,comma,SaleDate,MM/DD/YYYY,"Header1,Header2..."|T205,/path/file2.csv,semicolon,Date,YYYY-MM-DD,"HeaderA;HeaderB..."`  
  
Please correct the input and resubmit.  
```  
  
Data Transformation Logic:  
  
If the input is valid, perform the following data transformation steps:  
Initialize an empty list to hold standardized records and an empty list for data quality issues.  
  
For each pipe-separated FileDescription in the input string:  
Parse the TheaterID, FilePath, Delimiter, DateColumnName, SourceDateFormat, and CSVContent.  
  
Simulate reading the CSVContent string: Split into lines. Identify the header row (first line). Determine the column indices corresponding to the DateColumnName and the target output columns: Movie Title, Number of Tickets Sold, Concession Revenue, Staff On Duty. Assume standard header names like "Movie Title", "Movie", "Title", "Tickets Sold", "Tickets", "Concession Revenue", "Concessions", "Rev", "Staff On Duty", "Staff", "StaffCount". If essential columns are missing based on headers, add an issue note (`[TheaterID]: Missing essential columns`) and skip this file description.  
  
Iterate through the data rows (lines after the header):  
Let CurrentRowNumber be the index+2.  
Parse the row using the specified Delimiter. Handle potential errors if a row has incorrect number of fields (add issue note: `[TheaterID]: Row [CurrentRowNumber] has incorrect field count.` and skip row).  
  
Extract the raw values for Date, Movie Title, Tickets Sold, Concession Revenue, and Staff On Duty based on their determined column indices.  
  
Normalize Date: Attempt to parse the date value using the specified SourceDateFormat (handle DD-Mon-YY by assuming 20YY). Convert the parsed date into the user-specified target output date format. If parsing fails, mark the date as 'Invalid Date' and add issue note: `[TheaterID]: Row [CurrentRowNumber] failed date parsing.`.  
Normalize Movie Title: Trim whitespace. Apply specific mappings: "AM1" -> "Project Bolt"; "Comedy Movie 2" -> "Laugh Riot Returns"; "CM2" -> "Laugh Riot Returns".  
Normalize Numerics: Attempt to convert 'Tickets Sold' and 'Staff On Duty' to integers. Attempt to convert 'Concession Revenue' to a floating-point number. If conversion fails, mark the value as 'Invalid Number' and add issue note: `[TheaterID]: Row [CurrentRowNumber] failed numeric conversion for [Column Name].`. Assume Concession Revenue is in USD.  
  
Create a standardized record containing: TheaterID, Normalized Date, Normalized Movie Title, Normalized Tickets Sold, Normalized Concession Revenue (USD), Normalized Staff On Duty.  
Add the standardized record to the results list.  
  
After processing all file descriptions, prepare the final output.  
  
Output Format:  
  
Present the transformed and standardized data as a Markdown table.  
Before the table, explicitly state the target date format used and list any data quality issues encountered. If no issues, state "None".  
  
Example Output Structure:  
  
```markdown  
Target Date Format: [User-specified Target Date Format]  
  
Data Quality Issues:  
* [Issue 1 Description, e.g., T101: Row 3 skipped due to invalid number.]  
* [Issue 2 Description, e.g., T205: Date parsing failed for 1 row.]  
* (If no issues: None)  
  
| Theater ID | Date       | Movie Title         | Number of Tickets Sold | Concession Revenue (USD) | Staff On Duty |  
|------------|------------|---------------------|------------------------|--------------------------|---------------|  
| [TID]      | [Date]     | [Title]             | [Tickets]              | [Revenue]                | [Staff]       |  
| [TID]      | [Date]     | [Title]             | [Tickets]              | [Revenue]                | [Staff]       |  
| ...        | ...        | ...                 | ...                    | ...                      | ...           |  
  
```  
  
Constraints and Behavior:  
  
Process only the string data provided; do not assume access to actual files.  
Strictly follow the parsing, normalization, and standardization logic.  
Handle potential errors in CSV content or data types gracefully by marking data as invalid or noting skipped rows/files and specific issues.  
Use "USD" suffix for the Concession Revenue column header.  
Conclude the interaction politely.

**[user]**  
  
I need to process some theater data.

**[assistant]**  
  
Okay, I can help with that. First, please specify the desired date format for the final output table. Allowed formats are "YYYY-MM-DD", "MM/DD/YYYY", "DD-Mon-YYYY", or "DD/MM/YYYY".

**[user]**  
  
Use DD/MM/YYYY format. Here's the data: `T404,/path/data.csv,comma,Event Day,DD-Mon-YY,"Event Day,Title,Tickets,Revenue,Staff\n16-Apr-25,AM1,100,350.50,4\n17-Apr-25,CM2,120,410.00,4"`

**[assistant]**  
  
Target Date Format: DD/MM/YYYY  
  
Data Quality Issues:  
* None  
  
| Theater ID | Date | Movie Title | Number of Tickets Sold | Concession Revenue (USD) | Staff On Duty |  
|------------|------------|---------------------|------------------------|--------------------------|---------------|  
| T404 | 16/04/2025 | Project Bolt | 100 | 350.50 | 4 |  
| T404 | 17/04/2025 | Laugh Riot Returns | 120 | 410.00 | 4 |  
  
Data transformation complete. Let me know if you have more data to process.

**[user]**  
  
Yes, keep DD/MM/YYYY format. Process this: `T501,/path/log.csv,semicolon,LogDate,YYYY-MM-DD,"LogDate;Movie;Tickets Sold;Concession Revenue;Staff On Duty\n2025-04-18;New Film;Eighty;200.00;3\n2025-04-19-EXTRA;Old Film;70;180.00;3"`

**[assistant]**  
  
Target Date Format: DD/MM/YYYY  
  
Data Quality Issues:  
* T501: Row 2 failed numeric conversion for Tickets Sold.  
* T501: Row 3 failed date parsing.  
  
| Theater ID | Date | Movie Title | Number of Tickets Sold | Concession Revenue (USD) | Staff On Duty |  
|------------|------------|-------------|------------------------|--------------------------|---------------|  
| T501 | 18/04/2025 | New Film | Invalid Number | 200.00 | 3 |  
| T501 | Invalid Date| Old Film | 70 | 180.00 | 3 |  
  
Data transformation complete. Ready for the next one.

**[user]**  
  
Output format MM/DD/YYYY. Data: `T600,/path/x.csv,comma,Date,MM/DD/YYYY`

**[assistant]**  
  
**Error: Invalid Input Parameters**  
  
I could not process the request due to invalid input. Please ensure:  
1. You specified a valid target output date format ('YYYY-MM-DD', 'MM/DD/YYYY', 'DD-Mon-YYYY', or 'DD/MM/YYYY') before the data string.  
2. The data string contains one or more pipe-separated (|) source descriptions.  
3. Each source description has exactly 6 comma-separated values: TheaterID, FilePath, Delimiter ('comma' or 'semicolon'), DateColumnName, SourceDateFormat ('MM/DD/YYYY', 'YYYY-MM-DD', or 'DD-Mon-YY'), CSVContent (including header and at least one data row).  
  
The CSVContent for T600 appears to be missing or incomplete.  
  
Please correct the input and resubmit.

**[user]**  
  
Okay, sorry. Keep output MM/DD/YYYY. Corrected T600 data: `T600,/path/x.csv,comma,Date,MM/DD/YYYY,"Date,Movie,Tickets,Rev,StaffCount\n04/20/2025,SciFi Epic,150,550.75,6"`

**[assistant]**  
  
Target Date Format: MM/DD/YYYY  
  
Data Quality Issues:  
* None  
  
| Theater ID | Date | Movie Title | Number of Tickets Sold | Concession Revenue (USD) | Staff On Duty |  
|------------|------------|-------------|------------------------|--------------------------|---------------|  
| T600 | 04/20/2025 | SciFi Epic | 150 | 550.75 | 6 |  
  
Data transformation complete. Is there anything else?